# Chapter 04

## 1. MNIST
- 손글씨 숫자 이미지 집합
- 기계학습 분야에서 널리 사용되는 오픈소스 데이터셋
- Training Dataset : 60,000
- Test Dataset : 10,000
- 각 이미지는 28x28 픽셀
- 각 픽셀 : 0~255 사이의 값
- 각 이미지 별로 정답을 나타내는 레이블이 존재한다.

## 2. 구현하기
- ipynb 파일을 import 하기 위해서는 import_ipynb 패키지가 필요하다.
- pip install import_ipynb
- 사전에 구현해놓은 mnist.ipynb 파일을 사용한다.
- load_mnist() 함수를 호츨하면 인터넷에서 Dataset을 다운로드한다.
- 두번째 호출부터는 로컬에 저장된 pickle 파일을 읽어온다.

### 1) Data 모양 확인해보기

In [10]:
import import_ipynb
from mnist import load_mnist

# Dataset load
# 전처리 옵션들
# normalize : 이미지의 픽셀 값은 0.0~1.0 사이의 값으로 정규화 할지 여부
# flatten : 입력 이미지를 1차원 배열로 만들지 여부 (True : 784, False : 1x28x28)
# one_hot_label : one-hot encoding으로 저장할지 여부. True : [0,0,1,0,0,0,0,0,0], False : 2
(x_train, t_train), (x_test, t_test) = load_mnist(flatten = True, normalize=False)

print(x_train.shape) # (60000, 784)
print(t_train.shape) # (60000, )
print(x_test.shape)  # (10000, 784)
print(t_test.shape)  # (10000, )

(60000, 784)
(60000,)
(10000, 784)
(10000,)


### 2) 이미지 확인해보기

In [11]:
# 이미지 확인해보기
import numpy as np
from PIL import Image

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()
    
(x_train, t_train), (x_test, t_test) = load_mnist(flatten = True, normalize=False)

img = x_train[0]
label = t_train[0]
print(label) # 5

print(img.shape)          #(784,)
img = img.reshape(28, 28) # 원래 이미지 모양으로 변형
print(img.shape)          # (28, 28)

img_show(img)

5
(784,)
(28, 28)


### 3) 추론해보기

In [23]:
import Chapter_03 as af
import pickle

def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(flatten = True, normalize=False, one_hot_label=False)
    return x_test, t_test

def init_network():
    with open("sample_weight.pkl",'rb') as f: # 사전에 세팅해 놓은 가중치 값
        network = pickle.load(f)
        
    return network

def predict(network, x):
    # 은닉층 2층, 출력층 1층인 신경망
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x, W1) + b1
    z1 = af.relu(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = af.relu(a2)
    a3 = np.dot(z2, W3) + b3
    y= af.softmax(a3)
    
    return y

In [39]:
x, t = get_data()
network = init_network()

accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y) # 가장 큰 원소의 인덱스 리턴
    if p == t[i]:
        accuracy_cnt += 1

print("Accuracy:" + str(float(accuracy_cnt)/len(x)))

Accuracy:0.8286


## 3. Batch 처리
- Batch : 하나로 묶은 입력 데이터
- 데이터 하나당 배열 하나가 아니라 배열 하나에 데이터 여러개를 담고있다.
- 수치계산 라이브러리 대부분이 큰 배열을 효율적으로 처리할 수 있도록 최적화 되어있다.
- 데이터 전송의 병목현상을 줄여준다.
- 작은 것 여러번 보다 큰 것 한번 처리하는게 속도가 빠르다.

In [38]:
x, t = get_data()
network = init_network()

batch_size = 100 # 배치 크기
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1) # 1번째 차원을 축으로 각 원소들의 최대값을 찾음. 여기서는 각 행이 1차원.
    accuracy_cnt += np.sum(p == t[i:i+batch_size])

print("Accuracy:" + str(float(accuracy_cnt)/len(x)))

Accuracy:0.1003
